In [2]:
import numpy as np
import scipy
from clock_project.simulation.magnitude_quantification import calculate_information
import scipy
import json
from cogent3.util.deserialise import deserialise_object
import numpy as np
import os
import glob

import plotly.express as px
from cogent3.maths.measure import jsd




In [82]:
base_dir = '/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_model_fitting_350_threshold'
gene_paths = glob.glob(os.path.join(base_dir, '*/'))
alignment_dir = '/Users/gulugulu/repos/PuningAnalysis/data/ensembl_ortholog_sequences/homologies_alignment_common_name_350_threshold'
gene_aln_paths = glob.glob(os.path.join(alignment_dir, '*.json'))

# paths = '/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_alignment_350_threshold/'
# gene_paths = glob.glob(os.path.join(base_dir, '*/'))
# alignment_quality_dict = {}
# for gene_path in gene_paths:
#     gene_name = os.path.basename(gene_path.rstrip('/'))
#     aln_paths = glob.glob(os.path.join(paths, '*.json'))
#     for i in range(len(aln_paths)):
#         aln = deserialise_object(json.load(open(aln_paths[i], 'r')))
#         aln_sub = aln.take_seqs(aln.names[:2])
#         print(aln_sub.alignment_quality())

In [84]:
aln_gene = deserialise_object(json.load(open(gene_aln_paths[0], 'r')))

In [11]:
def change_nucleotide_order(motif_prob):
    nuc_freq = {'A': motif_prob[0],'C': motif_prob[1],'G': motif_prob[2],'T': motif_prob[3]}
    nuc_freq_reordered = {'T': nuc_freq['T'], 'C': nuc_freq['C'], 'A': nuc_freq['A'], 'G': nuc_freq['G']}
    return list(nuc_freq_reordered.values())


In [12]:
jsd([0.3, 0.2, 0.3, 0.2], [0.4, 0.5, 0.4, 0.5])

0.10615590677938203

In [14]:
change_nucleotide_order([0.2, 0.3, 0.2, 0.3])

[0.3, 0.3, 0.2, 0.2]

In [4]:
change_nucleotide_order([0.1882, 0.2884, 0.3106, 0.2128])

[0.2128, 0.2884, 0.1882, 0.3106]

In [5]:
def get_jsd_diff(triads_info):
    jsd_diff_dict = {}
    for identifier, info in triads_info.items():
        triads_info_value = info['triads_info_small_tree']
        triads_names = info['triads_species_names']
        nuc_freqs_dict = triads_info_value['nuc_freqs_dict']
        nuc_freq1 = change_nucleotide_order(nuc_freqs_dict['ingroup1'])
        nuc_freq2 = change_nucleotide_order(nuc_freqs_dict['ingroup2'])
        nuc_freq_internal_node = nuc_freqs_dict["internal_node"]
        jsd1 = jsd(nuc_freq1, nuc_freq_internal_node)
        jsd2 = jsd(nuc_freq2, nuc_freq_internal_node)
        jsd_diff = abs(jsd1 - jsd2)
        jsd_diff_dict[identifier] = jsd_diff
    return jsd_diff_dict
        

In [6]:
from scipy.stats import wasserstein_distance

def get_ingroup_jsd(triads_info):
    ingroup_jsd_dict = {}
    for identifier, info in triads_info.items():
        triads_info_value = info['triads_info_small_tree']
        jsd_dict = triads_info_value['jsd_dict']

        ingroup_jsd = jsd_dict['Ingroup_JSD']
        ingroup_jsd_dict[identifier] = ingroup_jsd
    return ingroup_jsd_dict

def get_ingroup_wst(triads_info):
    ingroup_wst_dict = {}
    for identifier, info in triads_info.items():
        triads_info_value = info['triads_info_small_tree']
        nuc_freqs_dict = triads_info_value['nuc_freqs_dict']
        ingroup1_freq = change_nucleotide_order(nuc_freqs_dict['ingroup1'])
        ingroup2_freq = change_nucleotide_order(nuc_freqs_dict['ingroup2'])
        ingroup_wst = wasserstein_distance(ingroup1_freq, ingroup2_freq)
        ingroup_wst_dict[identifier] = ingroup_wst

    return ingroup_wst_dict

def get_wst_diff(triads_info):
    wst_diff_dict = {}
    for identifier, info in triads_info.items():
        triads_info_value = info['triads_info_small_tree']
        nuc_freqs_dict = triads_info_value['nuc_freqs_dict']
        ingroup1_freq = change_nucleotide_order(nuc_freqs_dict['ingroup1'])
        ingroup2_freq = change_nucleotide_order(nuc_freqs_dict['ingroup2'])
        internal_node_freq = nuc_freqs_dict['internal_node']
        wst1 = wasserstein_distance(ingroup1_freq, internal_node_freq)
        wst2 = wasserstein_distance(ingroup2_freq, internal_node_freq)
        wst_diff = abs(wst1-wst2)
        wst_diff_dict[identifier] = wst_diff

    return wst_diff_dict

In [7]:
def get_ingroup_ens_diff(triads_info):
    ens_ingroup_dict = {}
    for identifier, info in triads_info.items():
        triads_info_value = info['triads_info_small_tree']
        triads_names = info['triads_species_names']
        ens_dict = triads_info_value['ens']
        ens_ingroup = abs(np.log(ens_dict[triads_names['ingroup1']]/ ens_dict[triads_names['ingroup2']]))
        ens_ingroup_dict[identifier] = ens_ingroup
    return ens_ingroup_dict

In [8]:
def get_ingroup_ens_absdiff(triads_info):
    ens_ingroup_dict = {}
    for identifier, info in triads_info.items():
        triads_info_value = info['triads_info_small_tree']
        triads_names = info['triads_species_names']
        ens_dict = triads_info_value['ens']
        ens_ingroup = abs(ens_dict[triads_names['ingroup1']]- ens_dict[triads_names['ingroup2']])
        ens_ingroup_dict[identifier] = ens_ingroup
    return ens_ingroup_dict


In [9]:
def get_ingroup_ens_Hellinger(triads_info):
    ens_ingroup_dict = {}
    for identifier, info in triads_info.items():
        triads_info_value = info['triads_info_small_tree']
        triads_names = info['triads_species_names']
        ens_dict = triads_info_value['ens']
        ens_ingroup = np.sqrt(2*(np.sqrt(ens_dict[triads_names['ingroup1']])- np.sqrt(ens_dict[triads_names['ingroup2']]))**2)
        ens_ingroup_dict[identifier] = ens_ingroup
    return ens_ingroup_dict

In [10]:
def get_nabla_absdiff(triads_info):
    nabla_diff_dict = {}
    for identifier, info in triads_info.items():
        triads_info_value = info['triads_info_small_tree']
        triads_names = info['triads_species_names']
        nabla_dict = triads_info_value['nabla_values']
        nbala_diff = abs(nabla_dict[triads_names['ingroup1']] - nabla_dict[triads_names['ingroup2']])
        nabla_diff_dict[identifier] = nbala_diff
    return nabla_diff_dict

In [11]:
def get_ingroup_nabla(triads_info):
    nabla_ingroup_dict = {}
    for identifier, info in triads_info.items():
        triads_info_value = info['triads_info_small_tree']
        triads_names = info['triads_species_names']
        nabla_dict = triads_info_value['nabla_values']
        ens_dict = triads_info_value['ens']
        nabla_difference = abs(np.log(nabla_dict[triads_names['ingroup1']]*ens_dict[triads_names['ingroup1']]/nabla_dict[triads_names['ingroup2']]*ens_dict[triads_names['ingroup2']]))
        nabla_ingroup_dict[identifier] = nabla_difference
    return nabla_ingroup_dict

In [12]:
alignment_length_dict = {}
alignment_dir_paths = glob.glob(os.path.join(alignment_dir, '*.json'))
for path in alignment_dir_paths:
    file_name = os.path.basename(path).rsplit('.', 1)[0]
    alignment = deserialise_object(json.load(open(path, 'r')))
    alignment_length_dict[file_name] = alignment.get_lengths()[0]


In [13]:
def remove_outliers_iqr(data1, data2):
    def compute_iqr_bounds(data):
        Q1 = np.percentile(data, 25)
        Q3 = np.percentile(data, 75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        return lower_bound, upper_bound

    # Calculate IQR bounds for both lists
    lower_bound1, upper_bound1 = compute_iqr_bounds(data1)
    lower_bound2, upper_bound2 = compute_iqr_bounds(data2)

    # Filter out pairs where either value is an outlier
    filtered_data1 = []
    filtered_data2 = []

    for val1, val2 in zip(data1, data2):
        if (lower_bound1 <= val1 <= upper_bound1) and (lower_bound2 <= val2 <= upper_bound2):
            filtered_data1.append(val1)
            filtered_data2.append(val2)

    return filtered_data1, filtered_data2

In [14]:
import os
import json
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Function to load JSON data
def load_json_data(path):
    with open(path, 'r') as file:
        return json.load(file)

# Function to compute required values
def compute_values(path):
    nabla_log_ratio_list = []
    ens_log_ratio_list = []

    triads_data_path = os.path.join(path, 'triads_info_dict.json')
    triads_info = load_json_data(triads_data_path)
    nabla_log_ratio_dict = get_ingroup_nabla(triads_info)
    ens_log_ratio_dict = get_ingroup_ens_diff(triads_info)
    ens_abs_diff_dict = get_ingroup_ens_absdiff(triads_info)
    jsd_diff_dict = get_jsd_diff(triads_info)
    ingroup_jsd_dict = get_ingroup_jsd(triads_info)
    ingroup_wst_dict  = get_ingroup_wst(triads_info)
    wst_diff_dict = get_wst_diff(triads_info)
    ens_hellinger_dict = get_ingroup_ens_Hellinger(triads_info)
    nabla_absdiff_dict = get_nabla_absdiff(triads_info)
    nabla_log_ratio_list = list(nabla_log_ratio_dict.values())
    ens_abs_diff_list = list(ens_abs_diff_dict.values())
    ens_log_ratio_list = list(ens_log_ratio_dict.values())
    jsd_diff_list = list(jsd_diff_dict.values())
    ingroup_jsd_list = list(ingroup_jsd_dict.values())
    ingroup_wst_list = list(ingroup_wst_dict.values())
    nabla_absdiff_list = list(nabla_absdiff_dict.values())
    ens_hellinger_list = list(ens_hellinger_dict.values())
    wst_diff_list = list(wst_diff_dict.values())

    return nabla_log_ratio_list, ens_log_ratio_list, ens_abs_diff_list, jsd_diff_list, ingroup_jsd_list, ingroup_wst_list, wst_diff_list, nabla_absdiff_list, ens_hellinger_list

# Initialize the dictionary to store the data
gene_data_dict = {}

JSD_difference_RMSE = []
ENS_difference_RMSE = []
# Populate the dictionary with data for each gene
for path in gene_paths:
    gene_name = os.path.basename(path.rstrip('/'))
    nabla_log_ratio_list, ens_log_ratio_list, ens_abs_diff_list, jsd_diff_list, ingroup_jsd_list, ingroup_wst_list, wst_diff_list, nabla_absdiff_list, ens_hellinger_list = compute_values(path)
    gene_data_dict[gene_name] = {
        'nabla_log_ratio': nabla_log_ratio_list,
        'ens_log_ratio': ens_log_ratio_list,
        'ens_abs_diff': ens_abs_diff_list,
        'jsd_diff': jsd_diff_list, 
        'ingroup_jsd': np.sqrt(ingroup_jsd_list),
        'ingroup_wst': ingroup_wst_list,
        'wst_diff': wst_diff_list,
        'nabla_absdiff': nabla_absdiff_list,
        'ens_hellinger': ens_hellinger_list
    }






In [45]:
gene_paths

['/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_model_fitting_350_threshold/ENSG00000138315/',
 '/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_model_fitting_350_threshold/ENSG00000143479/',
 '/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_model_fitting_350_threshold/ENSG00000117707/',
 '/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_model_fitting_350_threshold/ENSG00000155380/',
 '/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_model_fitting_350_threshold/ENSG00000183801/',
 '/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_model_fitting_350_threshold/ENSG00000149782/',
 '/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_model_fitting_350_threshold/ENSG00000107560/',
 '/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_model_fitting_350_threshold/ENSG00000110344/',
 '/Users/gulugulu/Desktop/honour

In [15]:
ens_list = []
for path in gene_paths:
    gene_name = os.path.basename(path.rstrip('/'))
    triads_data_path = os.path.join(path, 'triads_info_dict.json')
    triads_info = load_json_data(triads_data_path)
    for identifier, info in triads_info.items():
        ens_dict = info['triads_info_small_tree']['ens']
        for value in ens_dict.values():
            ens_list.append(value)
            if value > 1:
                print(gene_name, identifier)

    
    

ENSG00000117322 59
ENSG00000214655 240


In [16]:

correlation_list1 = []
p_value_list1 = []
for gene, lists in gene_data_dict.items():
    nabla_log_ratio_list, ens_log_ratio_list = remove_outliers_iqr(lists['nabla_log_ratio'], lists['ens_log_ratio'])
    #Add the correlation factor in the list
    cor, p_value = scipy.stats.pearsonr(np.abs(list(ens_log_ratio_list)), np.abs(list(nabla_log_ratio_list)))
    correlation_list1.append(cor)
    p_value_list1.append(p_value)

fig1 = px.box(correlation_list1, title= 'Pearson Correlation Test Nabla Log Ratio Vs. ENS Log Ratio')
# Update layout with labels and title
fig1.update_layout(
    template='plotly_white',
    margin=dict(l=20, r=20, t=50, b=20),
    autosize=True,
    yaxis_title_font={'size': 20},  
    xaxis_title_font={'size': 20}, 
    xaxis_title='Correlation Coefficient',
    yaxis_title='Value',
    width=None
)
fig1.show()
# fig1.write_image('nabla_ens_cc.pdf')

In [17]:
gene_names = list(gene_data_dict.keys())
# Create the bar chart
fig = go.Figure(go.Bar(
    x=correlation_list1,
    y=gene_names,
    orientation='h',  # Horizontal bar chart
    marker=dict(color=p_value_list1, coloraxis="coloraxis")  # Color based on the correlation factor
))
fig.add_shape(
    type="line",
    x0=0.3, y0=0, x1=0.3, y1=len(gene_names),
    line=dict(color="yellow", width=3, dash="dashdot"),
)

fig.add_shape(
    type="line",
    x0=-0.3, y0=0, x1=-0.3, y1=len(gene_names),
    line=dict(color="yellow", width=3, dash="dashdot"),
)

# Update layout for better visualization
fig.update_layout(
    xaxis_title="Correlation Coefficient",
    yaxis_title="Gene ID",
    coloraxis=dict(colorscale='Viridis'),  # Red for higher, blue for lower values
    height=1200  # Adjust height based on the number of genes to avoid squeezing
)

fig.show()
# fig.write_image('nabla_ens_cc_bar.pdf')

In [18]:
count = len([x for x in correlation_list1 if abs(x) > 0.3])
count

22

In [19]:
count = len([x for x in p_value_list1 if x < 0.05])
count

71

In [20]:
correlation_list4 = {}
p_value_list4 = {}
for gene, lists in gene_data_dict.items():
    wst_diff_list, ens_abs_diff_list = remove_outliers_iqr(lists['wst_diff'], lists['ens_abs_diff'])

    #Add the correlation factor in the list
    cor, p_value = scipy.stats.pearsonr(wst_diff_list, ens_abs_diff_list)
    correlation_list4[gene] = cor
    p_value_list4[gene] = p_value 

fig4 = px.box(correlation_list4.values(), title= 'Pearson Correlation Test WST Difference Vs. ENS difference')
# Update layout with labels and title
fig4.update_layout(
    template='plotly_white',
    margin=dict(l=20, r=20, t=50, b=20),
    autosize=True,
    yaxis_title_font={'size': 20},  
    xaxis_title_font={'size': 20}, 
    xaxis_title='Correlation Coefficient',
    yaxis_title='Value',
    width=None
)
fig4.show()
# fig2.write_image('jsd_ens_diff_cc.pdf')

In [21]:
gene_names = list(gene_data_dict.keys())
# Create the bar chart
fig = go.Figure(go.Bar(
    x=list(correlation_list4.values()),
    y=gene_names,
    orientation='h',  # Horizontal bar chart
    marker=dict(color=list(p_value_list4.values()), coloraxis="coloraxis")  # Color based on the correlation factor
))
fig.add_shape(
    type="line",
    x0=0.3, y0=0, x1=0.3, y1=len(gene_names),
    line=dict(color="yellow", width=3, dash="dashdot"),
)


# Update layout for better visualization
fig.update_layout(
    xaxis_title="Correlation Coefficient",
    yaxis_title="Gene ID",
    coloraxis=dict(colorscale='Viridis'),  # Red for higher, blue for lower values
    height=1200  # Adjust height based on the number of genes to avoid squeezing
)

fig.show()
# fig.write_image('jsd_ens_diff_cc_bar.pdf')

In [22]:
count = len([x for x in p_value_list4.values() if x < 0.05])
count


103

In [23]:
count = len([x for x in correlation_list4.values() if x > 0.3])
count

76

In [24]:
correlation_list2 = {}
p_value_list2 = {}
for gene, lists in gene_data_dict.items():
    jsd_diff_list, ens_abs_diff_list = remove_outliers_iqr(lists['jsd_diff'], lists['ens_abs_diff'])

    #Add the correlation factor in the list
    cor, p_value = scipy.stats.pearsonr(jsd_diff_list, ens_abs_diff_list)
    correlation_list2[gene] = cor
    p_value_list2[gene] = p_value 

fig2 = px.box(correlation_list2.values(), title= 'Pearson Correlation Test JSD Difference Vs. ENS difference')
# Update layout with labels and title
fig2.update_layout(
    template='plotly_white',
    margin=dict(l=20, r=20, t=50, b=20),
    autosize=True,
    yaxis_title_font={'size': 20},  
    xaxis_title_font={'size': 20}, 
    xaxis_title='Correlation Coefficient',
    yaxis_title='Value',
    width=None
)
fig2.show()
# fig2.write_image('jsd_ens_diff_cc.pdf')

In [25]:
gene_names = list(gene_data_dict.keys())
# Create the bar chart
fig = go.Figure(go.Bar(
    x=list(correlation_list2.values()),
    y=gene_names,
    orientation='h',  # Horizontal bar chart
    marker=dict(color=list(p_value_list2.values()), coloraxis="coloraxis")  # Color based on the correlation factor
))
fig.add_shape(
    type="line",
    x0=0.3, y0=0, x1=0.3, y1=len(gene_names),
    line=dict(color="yellow", width=3, dash="dashdot"),
)


# Update layout for better visualization
fig.update_layout(
    xaxis_title="Correlation Coefficient",
    yaxis_title="Gene ID",
    coloraxis=dict(colorscale='Viridis'),  # Red for higher, blue for lower values
    height=1200  # Adjust height based on the number of genes to avoid squeezing
)

fig.show()
# fig.write_image('jsd_ens_diff_cc_bar.pdf')

In [26]:
count = len([x for x in p_value_list2.values() if x < 0.05])
count

108

In [27]:
count = len([x for x in correlation_list2.values() if x > 0.3])
count

81

In [29]:
correlation_list3 = {}
p_value_list3 = {}
for gene, lists in gene_data_dict.items():
    ingroup_jsm_list, ens_abs_diff_list = remove_outliers_iqr(lists['ingroup_jsd'], lists['ens_abs_diff'])

    #Add the correlation factor in the list
    cor, p_value = scipy.stats.pearsonr(ingroup_jsm_list, ens_abs_diff_list)
    correlation_list3[gene] = cor
    p_value_list3[gene] = p_value 

fig3 = px.box(correlation_list3.values(), title= 'Pearson Correlation Test In-group JSD Vs. ENS difference')
# Update layout with labels and title
fig3.update_layout(
    template='plotly_white',
    margin=dict(l=20, r=20, t=50, b=20),
    autosize=True,
    yaxis_title_font={'size': 20},  
    xaxis_title_font={'size': 20}, 
    xaxis_title='Correlation Coefficient',
    yaxis_title='Value',
    width=None
)
fig3.show()
# fig2.write_image('jsd_ens_diff_cc.pdf')



In [30]:
# correlation_list3_subset = {gene: correlation_list3[gene] for gene in gene_list}
# p_value_list3_subset = {gene: p_value_list3[gene] for gene in gene_list}

In [31]:
count = len([x for x in p_value_list3.values() if x < 0.05])
count

120

In [32]:
count = len([x for x in correlation_list3.values() if x > 0.3])
count

87

In [33]:
gene_names = list(gene_data_dict.keys())
# Create the bar chart
fig = go.Figure(go.Bar(
    x=list(correlation_list3.values()),
    y=gene_names,
    orientation='h',  # Horizontal bar chart
    marker=dict(color=list(p_value_list3.values()), coloraxis="coloraxis")  # Color based on the correlation factor
))
fig.add_shape(
    type="line",
    x0=0.3, y0=0, x1=0.3, y1=len(gene_names),
    line=dict(color="yellow", width=3, dash="dashdot"),
)


# Update layout for better visualization
fig.update_layout(
    xaxis_title="Correlation Coefficient",
    yaxis_title="Gene ID",
    coloraxis=dict(colorscale='Viridis'),  # Red for higher, blue for lower values
    height=1200  # Adjust height based on the number of genes to avoid squeezing
)

fig.show()
# fig.write_image('ingroup_jsd_ens_cc_bar.pdf')

In [34]:
ancester_distirbution_dict = json.load(open('/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_350_threshold/internal_root_distributions.json', 'r'))

information_dict = {}
for gene_id, distirbutions_info in ancester_distirbution_dict.items():
    information_dict[gene_id] = {}
    for identifier, distirbution in distirbutions_info.items():
        information_dict[gene_id][identifier] = calculate_information(distirbution)

information_list = [value for gene_id in information_dict for value in information_dict[gene_id].values()]
average_information_dict = {}
for gene, informations in information_dict.items():
    average_information_dict[gene] = np.average(list(informations.values()))



In [35]:
# import plotly.express as px
# corr_dict = {gene: correlation_list3[gene] for gene in average_information_dict.keys()}
# information_correlation_fig = px.scatter(x = list(average_information_dict.values()), y = list(corr_dict.values()), labels={'x':'Information (Average)', 'y':'Correlation Coefficient'}, trendline="ols", title= None)
# # Update layout with labels and title
# information_correlation_fig.update_layout(
#     template='plotly_white',
#     margin=dict(l=20, r=20, t=50, b=20),
#     autosize=True,
#     yaxis_title_font={'size': 20},  
#     xaxis_title_font={'size': 20}, 
#     width=None 
# )
# information_correlation_fig.show()

In [36]:


def plot_gene_data(gene_data_dict, xcol, ycol):
    keys = list(gene_data_dict.keys())
    rows = int(len(keys) ** 0.5) + 1  # Calculate the number of rows for subplots
    cols = (len(keys) + rows - 1) // rows  # Calculate the number of columns

    fig = make_subplots(rows=rows, cols=cols, subplot_titles=[f'{key}' for key in keys])
    
    # Populate subplots
    for index, key in enumerate(keys, start=1):
        gene_data = gene_data_dict[key]
        x_value, y_value = remove_outliers_iqr(gene_data[xcol], gene_data[ycol])

        row = (index - 1) // cols + 1
        col = (index - 1) % cols + 1
        
        fig.add_trace(
            go.Scatter(
                x=x_value,
                y=y_value,
                mode='markers',
                name=f'{key}'
            ),
            row=row,
            col=col
        )
        
        # Adding a trend line
        fig.add_trace(
            go.Scatter(
                x=x_value,
                y=np.poly1d(np.polyfit(x_value, y_value, 1))(x_value),
                mode='lines',
                name=f'Trend {key}',
                line=dict(color='red')
            ),
            row=row,
            col=col
        )
        
        # Update axis properties
        fig.update_xaxes(title_text=xcol if row == rows else "", row=row, col=col)
        fig.update_yaxes(title_text=ycol if col == 1 else "", row=row, col=col)
    
    fig.update_layout(
        height=300 * rows,  # Set a reasonable height based on the number of rows
        width=300 * cols,   # Set a reasonable width based on the number of columns
        showlegend=False
    )
    
    return fig

In [37]:
# Usage example
fig = plot_gene_data(gene_data_dict, 'nabla_log_ratio', 'ens_log_ratio')

fig.update_layout(
title_text="Scatter Plots of Ingroup JSD vs. ENS Difference",)
fig.show()

In [38]:
path = '/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_model_fitting_350_threshold/ENSG00000065613'


In [39]:
triads_data_path = os.path.join(path, 'triads_info_dict.json')
triads_info = load_json_data(triads_data_path)
ens_ingroup_list = []
ingroup_jsd_list = []
for identifier, info in triads_info.items():
    triads_info_value = info['triads_info_small_tree']
    triads_names = info['triads_species_names']
    ingroup_jsd = triads_info_value['jsd_dict']['Ingroup_JSD']
    ens_dict = triads_info_value['ens']
    ens_ingroup = abs(ens_dict[triads_names['ingroup1']] - ens_dict[triads_names['ingroup2']])
    ens_ingroup_list.append(ens_ingroup)
    ingroup_jsd_list.append(ingroup_jsd)


ens_ingroup_list2, ingroup_jsd_list2 = remove_outliers_iqr(ens_ingroup_list, ingroup_jsd_list)


In [87]:
indices_ens_diff2 = list(range(1, len(ens_ingroup_list2) + 1))
indices_ingroup_jsd2 = list(range(1, len(ingroup_jsd_list2) + 1))

list_pair = []
for i in range(len(indices_ens_diff2)):
    jsd_value, ens = ingroup_jsd_list2[i], ens_ingroup_list2[i]
    list_pair.append((jsd_value, ens))

# Create the scatter plot
fig1 = go.Figure()

# Add ENS Differences scatter plot
fig1.add_trace(go.Scatter(
    x=indices_ingroup_jsd2, y=ens_ingroup_list2, mode='markers',
    marker=dict(size=4), name='ENS Differences'))

fig1.add_trace(go.Scatter(
    x=indices_ingroup_jsd2, y=np.sqrt(ingroup_jsd_list2), mode='markers',
    marker=dict(size=4), name='Ingroup JSM'))


# Update layout for clear visualization
fig1.update_layout(
    title='Uniform distirbution of each property',
    xaxis_title='Index',
    yaxis_title='Values',
    showlegend=True,
    
)

fig1.show()

In [41]:
sorted_data = sorted(list_pair, key=lambda x: x[1])

In [42]:
x_values = [x[0] for x in sorted_data]
y_values = [x[0] for x in sorted_data]

In [43]:
# Create the scatter plot
fig1 = go.Figure()

# Add ENS Differences scatter plot
fig1.add_trace(go.Scatter(
    x=np.sqrt(x_values), y=sorted(np.sqrt(y_values)), mode='markers',
    marker=dict(size=4), name='ENS Differences'))



# Update layout for clear visualization
fig1.update_layout(
    title='Uniform distirbution of each property',
    xaxis_title='Index',
    yaxis_title='Values',
    showlegend=True,
    
)

fig1.show()

In [44]:
import numpy as np
import scipy.stats as stats

def permute_test_correlation(x, y, n_permutations=10000):
    # Calculate the actual correlation
    actual_corr, _ = stats.pearsonr(x, y)
    
    # To hold the permuted correlations
    permuted_corrs = []

    original_y = np.copy(y)
    
    # Permutation test
    for _ in range(n_permutations):
        # Shuffle one of the lists
        np.random.shuffle(y)
        # Compute the correlation of the permuted data
        perm_corr, _ = stats.pearsonr(x, y)
        permuted_corrs.append(perm_corr)

        y[:] = original_y
    
    # Compute p-value: proportion of permuted correlations as extreme as the actual one
    p_value = np.sum(np.abs(permuted_corrs) >= np.abs(actual_corr)) / n_permutations
    
    return actual_corr, p_value, permuted_corrs



